In [1]:


import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))



2023-09-02 12:27:42.489764: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-02 12:27:42.521483: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-02 12:27:42.522952: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-02 12:27:43.087127: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\u00C0-\u017F"          #latin
                           u"\u2000-\u206F"          #generalPunctuations
                               
                           "]+", flags=re.UNICODE)
    english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    #latin_pattern=re.compile('[A-Za-z\u00C0-\u00D6\u00D8-\u00f6\u00f8-\u00ff\s]*',)
    
    text=emoji_pattern.sub(r'', text)
    text=english_pattern.sub(r'', text)

    return text

def remove_punctuations(my_str):
    # define punctuation
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰⚽️✌�￰৷￰'''
    
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct



def joining(text):
    out=' '.join(text)
    return out

def preprocessing(text):
    out=remove_punctuations(replace_strings(text))
    return out

In [3]:
train_url = 'https://raw.githubusercontent.com/SyedT1/NLP/main/FromScratch/error%20detection/data/data/train_data.csv'
test_url = 'https://raw.githubusercontent.com/SyedT1/NLP/main/FromScratch/error%20detection/data/data/test_data.csv'
df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)
stop_words_df = pd.read_excel('stopwords_bangla.xlsx',index_col=False)
STOPWORDS = set([word.strip() for word in stop_words_df['words']])

In [4]:
df_train['Comment'] = df_train.Comment.apply(lambda x: preprocessing(str(x)))
df_test['Comment'] = df_test.Comment.apply(lambda x:preprocessing(str(x)))
df = pd.concat([df_train,df_test],ignore_index = True)

In [5]:
set(df['Category'].values)

{'Code Switching', 'Grammatical', 'Multiple Errors', 'Spelling', nan}

In [6]:
def encode(s):
    d = {
        "Code Switching":0,
        "Grammatical":1,
        "Multiple Errors":2,
        "Spelling":3
    }
    if s in d:
        return s
    else:
        return 4
df['Category'] = df.Category.apply(lambda x: encode(x))

In [7]:
tokenizer = Tokenizer(num_words=50000, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(df['Comment'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 12986 unique tokens.


In [8]:
X = tokenizer.texts_to_sequences(df['Comment'].values)
X = pad_sequences(X, maxlen=3000)

In [9]:
Y = pd.get_dummies(df['Category'],columns=df["Category"]).values
Y

array([[0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0]], dtype=uint8)

In [17]:
lm = round(len(df)*0.80)
X_train, X_test, Y_train, Y_test = X[:lm],X[lm:],Y[:lm],Y[lm:]

In [18]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)


(8038, 3000) (8038, 5)
(2009, 3000) (2009, 5)


In [19]:
model=Sequential()
model.add(Embedding(50000,100,input_length=3000))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [20]:
history = model.fit(X_train, Y_train, epochs=7, batch_size=64,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])



Epoch 1/7
114/114 [==============================] - 230s 2s/step - loss: 1.1641 - accuracy: 0.5687 - val_loss: 1.0963 - val_accuracy: 0.5808
Epoch 2/7
114/114 [==============================] - 253s 2s/step - loss: 0.9464 - accuracy: 0.6475 - val_loss: 1.0636 - val_accuracy: 0.6306
Epoch 3/7
114/114 [==============================] - 260s 2s/step - loss: 0.6381 - accuracy: 0.7729 - val_loss: 1.0802 - val_accuracy: 0.6405
Epoch 4/7
114/114 [==============================] - 290s 3s/step - loss: 0.3729 - accuracy: 0.8728 - val_loss: 1.1387 - val_accuracy: 0.6493
Epoch 5/7
114/114 [==============================] - 266s 2s/step - loss: 0.2488 - accuracy: 0.9197 - val_loss: 1.3659 - val_accuracy: 0.6443


In [28]:
accr = model.evaluate(X_test,Y_test)

y_pred = model.predict(X_test)

# Calculate macro recall
macro_recall = recall_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
macro_precision = precision_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
# Print the results
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n  Macro Recall: {:0.3f}\n  Macro Precision: {:0.3f}'.format(accr[0], accr[1], macro_recall, macro_precision))

63/63 [==============================] - 15s 241ms/step
Test set
  Loss: 1.361
  Accuracy: 0.627
  Macro Recall: 0.467
  Macro Precision: 0.552
